# Component examples:
This notebook demonstrates some of the main components in the OPPM pipeline, such as encoding and different encoders, the labeling function & the concept of feature rollback. To reference our implementation of the custom converters, please refer to this river GitHub issue thread: https://github.com/online-ml/river/issues/582.

In [1]:
from datetime import datetime
from river import feature_extraction as fx
from river import compose

from OPPM.components import *
from OPPM.river_additions import converters, encoders

## Encoders

In PPM, particularly in online PPM, encoders are essential to representing an entire case as an *encoded trace*. Since we want to make predictions for an entire case and not just for one activity, we need to learn the properties of that entire case rather than the incoming data point. To demonstrate this, we've defined a (very simplified) event log example in dictionary shape, which is what river expects and supports.

In [2]:
dataset = [
    {'Case ID': 1, 'Activity': 'request', 'Resource': 'customer', 'Timestamp': datetime(2021, 1, 1, 0, 0), 'Amount': 100, 'Credit_Score': None},
    {'Case ID': 2, 'Activity': 'request', 'Resource': 'customer', 'Timestamp': datetime(2021, 1, 2, 0, 0), 'Amount': 4000, 'Credit_Score': None},
    {'Case ID': 2, 'Activity': 'check credit score', 'Resource': 'employee_1', 'Timestamp': datetime(2021, 1, 3, 0, 0), 'Amount': None, 'Credit_Score': 750},
    {'Case ID': 1, 'Activity': 'change', 'Resource': 'customer', 'Timestamp': datetime(2021, 1, 3, 0, 0), 'Amount': 200, 'Credit_Score': None},
    {'Case ID': 1, 'Activity': 'check credit score', 'Resource': 'employee_1', 'Timestamp': datetime(2021, 1, 3, 0, 0), 'Amount': None, 'Credit_Score': 400},
    {'Case ID': 2, 'Activity': 'accepted', 'Resource': 'employee_2', 'Timestamp': datetime(2021, 1, 5, 0, 0), 'Amount': None, 'Credit_Score': None},
    {'Case ID': 1, 'Activity': 'denied', 'Resource': 'employee_2', 'Timestamp': datetime(2021, 1, 5, 0, 0), 'Amount': None, 'Credit_Score': None},
]

With our dataset defined, we can loop through the dataset in a regular *for loop*. Note: in a traditional setting where a *y* is given, each dataset item would be a tuple with (x, y). In the case of PPM there is no label column, thus only x is given. River expects a *y* column however, so in that case we simply change our for loop to 'for x, _ in dataset', and ignore the y.

In [3]:
for x in dataset:
    print(x)

{'Case ID': 1, 'Activity': 'request', 'Resource': 'customer', 'Timestamp': datetime.datetime(2021, 1, 1, 0, 0), 'Amount': 100, 'Credit_Score': None}
{'Case ID': 2, 'Activity': 'request', 'Resource': 'customer', 'Timestamp': datetime.datetime(2021, 1, 2, 0, 0), 'Amount': 4000, 'Credit_Score': None}
{'Case ID': 2, 'Activity': 'check credit score', 'Resource': 'employee_1', 'Timestamp': datetime.datetime(2021, 1, 3, 0, 0), 'Amount': None, 'Credit_Score': 750}
{'Case ID': 1, 'Activity': 'change', 'Resource': 'customer', 'Timestamp': datetime.datetime(2021, 1, 3, 0, 0), 'Amount': 200, 'Credit_Score': None}
{'Case ID': 1, 'Activity': 'check credit score', 'Resource': 'employee_1', 'Timestamp': datetime.datetime(2021, 1, 3, 0, 0), 'Amount': None, 'Credit_Score': 400}
{'Case ID': 2, 'Activity': 'accepted', 'Resource': 'employee_2', 'Timestamp': datetime.datetime(2021, 1, 5, 0, 0), 'Amount': None, 'Credit_Score': None}
{'Case ID': 1, 'Activity': 'denied', 'Resource': 'employee_2', 'Timestamp': 

Since the data is defined as a dictionary, it is trivial to look at single values (such as the loan amount) only by simply indexing the x.

In [4]:
for x in dataset:
    if x['Case ID'] == 1: print(x['Activity'], x['Amount'])

request 100
change 200
check credit score None
denied None


As we can see, the loan amount is *changed* from 100 to 200 for case id 1 with the second activity. Additionally, we can see that by the sparse nature of an event log, the loan amount is not repeatedly stored with each data instance, rather only included when the value is either added or changed. In practical terms, this means that if we simply iterate over our dataset and only look at the given data instances, whenever the 'check credit score' activity is performed, the 'Amount' feature is not given in that data instance. Thus, we look at real-time trace encoding.

River's base *stats* module includes some basic feature transformers. While these can be used as encoding methods (the *agg* methods), they do not support the two main and most basic encoding methods: storing the *first* or the *last* value for a feature. Therefore, they, among others, can be found in the *encoders.py* script in the *river_additions* folder, but for explanation sake we have repeated their definition in this file.

They are essentially opposite in their working: the *last* state encoder looks for a new value that is not None or an empty string, and if it is neither, it saves that new value as the last seen value. The *first* state encoder instead checks whether the first value has been defined already (a non-NA value is seen), and if it has, it simply returns that value. Otherwise, if the current value for x is not an empty string or None type, that current value is stored as the first value.

In [5]:
from river.stats import base

class Last(base.Univariate):
    def __init__(self):
        self.last_value = None
    
    def update(self, x = None):
        if x != "" and x != None:
            self.last_value = x
        return self
    
    def get(self):
        return self.last_value

class First(base.Univariate):
    def __init__(self):
        self.first_value = None
    
    def update(self, x = None):
        if self.first_value == None and x != "" and x != None: 
            self.first_value = x
        return self

    def get(self):
        return self.first_value

For our example, we always want to know the most up-to-date value for *amount*, but we can just look at the first known value for the credit score within that case id. For this, we define multiple encoders, and combine them in a TransformerUnion. If we call this TransformerUnion 'encodings', we can see the river implementation.

In [6]:
last_credit = fx.Agg(on = 'Credit_Score', by = 'Case ID', how = Last())
first_amount = fx.Agg(on = 'Amount', by = 'Case ID', how = First())
encodings = compose.TransformerUnion(last_credit, first_amount)
encodings

TransformerUnion (
  Agg (
    on="Credit_Score"
    by=['Case ID']
    how=Last
  ),
  Agg (
    on="Amount"
    by=['Case ID']
    how=First
  )
)

With our encoders defined, we can again iterate through the dataset and call 'encodings.learn_one(x)' to update the encoders with the newer value. By calling 'encodings.transform_one(x)' we can retrieve this value. 

In [7]:
for x in dataset:
    print(x)
    print(encodings.transform_one(x))
    print("\n")
    encodings.learn_one(x)

{'Case ID': 1, 'Activity': 'request', 'Resource': 'customer', 'Timestamp': datetime.datetime(2021, 1, 1, 0, 0), 'Amount': 100, 'Credit_Score': None}
{'Amount_first_by_Case ID': None, 'Credit_Score_last_by_Case ID': None}


{'Case ID': 2, 'Activity': 'request', 'Resource': 'customer', 'Timestamp': datetime.datetime(2021, 1, 2, 0, 0), 'Amount': 4000, 'Credit_Score': None}
{'Amount_first_by_Case ID': None, 'Credit_Score_last_by_Case ID': None}


{'Case ID': 2, 'Activity': 'check credit score', 'Resource': 'employee_1', 'Timestamp': datetime.datetime(2021, 1, 3, 0, 0), 'Amount': None, 'Credit_Score': 750}
{'Amount_first_by_Case ID': 4000, 'Credit_Score_last_by_Case ID': None}


{'Case ID': 1, 'Activity': 'change', 'Resource': 'customer', 'Timestamp': datetime.datetime(2021, 1, 3, 0, 0), 'Amount': 200, 'Credit_Score': None}
{'Amount_first_by_Case ID': 100, 'Credit_Score_last_by_Case ID': None}


{'Case ID': 1, 'Activity': 'check credit score', 'Resource': 'employee_1', 'Timestamp': datetime

This encoding inhibits the behaviour that we want: we first transform the current data instance (giving us, for example, the case id) and see the current encoded trace. Whenever we call learn_one, it includes the current data instances' to update the encoder. Note that if we flip transform_one and learn_one, we immediately include the current trace's data. For PPM, the behaviour we want is to *include* the current data's information when predicting, but to *exclude* the current data when we train the model, since this has too much information (we want to predict before we know the outcome, not as we've already learned it). 

Note that the *first* encoder is not update when data is overwritten, after the amount is changed in the fourth data instance, the initial amount of 100 is still seen in the encoder for the activity after that. Naturally, if we want the data to be overwritten with this newer information (or store both), we could just us a different encoder.

Lastly, we also want to be able to remove an encoders' contents once the label is known. There is no neat implementation of this in River, therefore we utilize a crude method which deletes the contents. We acknowledge that there is presumably a better way of doing this. 

Each encoder's group needs to be individually deleted, so for TransformerUnions, we loop through every transformer in the union. For single transformers this loop is not needed.

In [ ]:
#if label is known for x and the model has been trained:
for name in encodings.transformers:
    encoder = encodings[name]
    del encoder.groups[encoder._get_key(x)]

This concludes the demonstration of the working of encoders, all cuerrently supported encoders can be found in components/encoding.py, under 'get_encoder()'.

## Labeling function
Since our data does not contain a *y* variable, we need some function to determine the label. One of the most common and useful functions to determine a label in PPM is a value-occurence labeling function, a real-time implementation of which is given below.

In [8]:
class labeler():

    def __init__(self, outcomes:set, positive_outcomes:str, feature:str, positive_label=1, negative_label=0):
        self.outcomes = outcomes
        self.positive_outcomes = positive_outcomes
        self.feature = feature
        self.positive_label = positive_label
        self.negative_label = negative_label

    def check(self, x):
        if x[self.feature] in self.outcomes:
            return True
        else:
            return False
    
    def get(self, x):
        return self.positive_label if x[self.feature] in self.positive_outcomes else self.negative_label

This function contains two functions: check and get. The check function uses fast set comprehension to determine if the value for the feature which contains the outcome is in the set of outcomes (i.e., the outcome is present in the current data instance) and the get function then determines the final outcome.

To use the labeling function, it first needs to be initiated as below.

In [9]:
label = labeler(
    outcomes = {'accepted', 'denied', 'cancelled'},
    positive_outcomes = {'accepted'},
    feature = 'Activity',
    positive_label = 1,
    negative_label = 0
)

Now, we can again loop through the data and print the outcome for each row.

In [10]:
for x in dataset:
    check = label.check(x)
    print(f'Label found in data instance: {check}')
    if check:
        print(f'Found label: {label.get(x)}')
        #model would train here

Label found in data instance: False
Label found in data instance: False
Label found in data instance: False
Label found in data instance: False
Label found in data instance: False
Label found in data instance: True
Found label: 1
Label found in data instance: True
Found label: 0


As expected, we get no labels for the first 5 activities since 'accepted', 'denied' or 'cancelled' is not in the 'Activity' column, but we get a label for the last two activities: 'accepted' and 'denied', corresponding to the 0 and 1 label.

This concludes the demonstration of the labeling function. More advanced labeling functions can of course use these two simple functions as well, but are currently not implemented. 

## Feature rollback
With feature rollback, we refer to the idea of showing a classifier the full history of encoded traces at training time to help it generalize to incomplete traces. Without rollback, the encoded trace as shown in the *encoders* section of this notebook would be available to the classifier once the label from the labeling function is known, which would give it essentially the most information possible. However, when using this model to predict with an incomplete trace, a lot of missing values would be introducted which the model would not be able to generalize very well. 

Our implementation is really quite simple: we simply store the transform_one(x) for each iteration and loop through it at training time. After the model has been trained, these features are no longer needed and thus deleted.

In [ ]:
# training step, after label.check() == True
y = label.get(x)
for version in predictors[case_id]:
    model.learn_one(version, y)
del predictors[case_id]

# prediction step, if label.check() == False
X = encodings.learn_one(x).transform_one(x) #encode

predictors[case_id].append(X) #add trace to predictors

predictions[case_id].append(model.predict_proba_one(X)[label.positive_label]) #predict